# Visualization: Trading Session

In [159]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [183]:
from trading_bot.agent import Agent

model_name = 'model_debug_50'
test_stock = 'data/India/BajajFinanceFTest.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

In [184]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2018-01-01,1715.731567
1,2018-01-02,1714.737305
2,2018-01-03,1705.440918
3,2018-01-04,1743.918579
4,2018-01-05,1804.070801


### 3. Running Eval

In [185]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-09-24 19:02:54 Tornado root[3847] DEBUG switching to TensorFlow for CPU
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1715.73
2020-09-24 19:02:54 Tornado root[3847] DEBUG Sell at: $1714.74 | Position: -$0.99
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1705.44
2020-09-24 19:02:54 Tornado root[3847] DEBUG Sell at: $1804.07 | Position: +$98.63
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1791.84
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1770.12
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1759.93
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1728.16
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1677.65
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1691.42
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1651.15
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1692.12
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1696.54
2020-09-24 19:02:54 Tornado root[3847] DEBUG Buy at: $1698.88
2020-09-24 19:

2020-09-24 19:02:55 Tornado root[3847] DEBUG Buy at: $2711.80
2020-09-24 19:02:55 Tornado root[3847] DEBUG Buy at: $2673.01
2020-09-24 19:02:55 Tornado root[3847] DEBUG Buy at: $2670.47
2020-09-24 19:02:55 Tornado root[3847] DEBUG Buy at: $2685.36
2020-09-24 19:02:55 Tornado root[3847] DEBUG Buy at: $2736.25
2020-09-24 19:02:55 Tornado root[3847] DEBUG Buy at: $2801.79
2020-09-24 19:02:55 Tornado root[3847] DEBUG Sell at: $2818.72 | Position: +$106.92
2020-09-24 19:02:55 Tornado root[3847] DEBUG Sell at: $2797.11 | Position: +$124.10
2020-09-24 19:02:55 Tornado root[3847] DEBUG Sell at: $2749.25 | Position: +$78.78
2020-09-24 19:02:55 Tornado root[3847] DEBUG Buy at: $2818.77
2020-09-24 19:02:55 Tornado root[3847] DEBUG Sell at: $2845.06 | Position: +$159.70
2020-09-24 19:02:55 Tornado root[3847] DEBUG Sell at: $2839.68 | Position: +$103.43
2020-09-24 19:02:55 Tornado root[3847] DEBUG Sell at: $2828.28 | Position: +$26.49
2020-09-24 19:02:55 Tornado root[3847] DEBUG Sell at: $2860.20 |

### 4. Visualize

In [186]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [187]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

In [188]:
cap = [0]
inv = 0
ret = 0
for i in range(len(df)):
    if df.iloc[i]['action']=='BUY':
        cap.append(cap[-1]+df.iloc[i]['actual'])
        inv+=df.iloc[i]['actual']
    if df.iloc[i]['action']=='SELL':
        cap.append(cap[-1]-df.iloc[i]['actual'])
        ret += df.iloc[i]['actual']

In [189]:
req_cap = max(cap)
req_cap

26209.286251999998

In [190]:
prof = ret+(df['action'].value_counts()['BUY']-df['action'].value_counts()['SELL'])*df.iloc[-1]['actual']-inv

In [191]:
(prof/ req_cap)*100

20.966800111088464

In [192]:
df['action'].value_counts()

BUY     101
SELL     92
HOLD     52
Name: action, dtype: int64

In [118]:
position = [history[0][0]] + [x[0] for x in history]

In [149]:
df

,date,actual,position,action
0,2018-01-01,1715.731567,1715.731567,HOLD
1,2018-01-02,1714.737305,1715.731567,BUY
2,2018-01-03,1705.440918,1714.737305,SELL
3,2018-01-04,1743.918579,1705.440918,BUY
4,2018-01-05,1804.070801,1743.918579,HOLD
5,2018-01-08,1826.292358,1804.070801,SELL
6,2018-01-09,1809.042114,1826.292358,HOLD
7,2018-01-10,1791.841553,1809.042114,HOLD
8,2018-01-11,1770.117065,1791.841553,BUY
9,2018-01-12,1759.926025,1770.117065,BUY


4233.569336